[View in Colaboratory](https://colab.research.google.com/github/saranyamandava/Lambda-School-DataScience/blob/master/Week7__Model_Tuning_Assignment_2.ipynb)

# Part One: K-fold Cross Validation

The challenge of training machine learning models is to be able to make accurate predictions on previously unseen real-world data in spite of the fact that we only have a finite training dataset to learn from. 

One way of validating our model's quality-of-fit and avoiding overfitting/underfitting, is to use the test_train_split method like we did in the code challenge. With this method, the randomly selected test dataset can be used to evaluate how our model performs on data that it has not yet seen in the training process. However, there are downsides to this approach:

*   We lose a valuable portion of data that we would prefer to be able to train on to serve as the test dataset. We would prefer to have both the testing and training datasets be as large as possible.
*   With small datasets, measures of our model's quality using the test_train_split method often have a high variance. (We saw this behavior when we changed the random seed in the code challenge)

We can reduce the severity of both of these drawbacks by using what is called K-fold Cross Validation:

[Short Video Explaining K-Fold Cross Validation](https://www.youtube.com/watch?v=TIgfjmp-4BA)

[How to Implement K-Fold Cross Validation on the Pima Indians Diabetes dataset](https://machinelearningmastery.com/evaluate-performance-machine-learning-algorithms-python-using-resampling/)

## DO THIS:

**1)** Train a logistic regression model on the titanic dataset predicting survivors first using a 20-80% test_train_split and print the accuracy of your model using 5 different random seeds.

**2)** Use 5-fold Cross Validation on the titanic dataset. Print out the accuracies from each of the 5 folds of the cross validation, then print the final mean and standard deviation of those cross validation accuracies. How do the accuracies on each of the inidvidual folds compare to the accuracies of the test_train_split approach? Is the variance in accuracies of the cross-validation approach higher or lower than the variance of the test_train_split approach? 

**3)** Try using 3-fold Cross Validation as well as 10-fold cross validation. How does the number of folds in the cross-validation process affect the outcome? How many folds should be used?

---
I would give you more boilerplate code here, but I don't want to make it too easy. The articles linked above should be sufficient for this purpose.

In [71]:
##### YOUR CODE HERE ##### - Feel free to add code cells as necessary.

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split,KFold,RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LogisticRegression,Ridge
from itertools import chain, combinations
from sklearn import model_selection


titanic = sns.load_dataset('titanic')

# drop duplicate/analogous columns
titanic = titanic.drop(['alive',
                        'adult_male',
                        'who',
                        'class',
                        'embark_town'], axis=1)

# take care of missing data
titanic['embarked'] = titanic['embarked'].fillna(method='ffill')
titanic = titanic.drop(['deck'], axis=1)
titanic['age'] = titanic['age'].fillna(method='ffill')
titanic = titanic.drop_duplicates()

for label in ['embarked','sex', 'alone']:
    titanic[label] = LabelEncoder().fit_transform(titanic[label])

embarked_one_hot = OneHotEncoder().fit_transform(titanic[['embarked']]).toarray()
embarked = pd.DataFrame(embarked_one_hot, 
                        columns=['Southampton', 'Cherbourg', 'Queenstown'], 
                        dtype=np.int64)

titanic = titanic.reset_index(drop=True)
data_encoded = titanic.join([embarked])
data_encoded = data_encoded.drop(['embarked'], axis=1)

print (data_encoded.head())

print(data_encoded.shape)

   survived  pclass  sex   age  sibsp  parch     fare  alone  Southampton  \
0         0       3    1  22.0      1      0   7.2500      0            0   
1         1       1    0  38.0      1      0  71.2833      0            1   
2         1       3    0  26.0      0      0   7.9250      1            0   
3         1       1    0  35.0      1      0  53.1000      0            0   
4         0       3    1  35.0      0      0   8.0500      1            0   

   Cherbourg  Queenstown  
0          0           1  
1          0           0  
2          0           1  
3          0           1  
4          0           1  
(833, 11)


In [72]:

random_seed_values = [7,42,43,91,52]

for random_seed in random_seed_values:
  x_train, x_test, y_train, y_test = train_test_split(
    data_encoded.drop(['survived', 'Southampton'], axis=1), 
    data_encoded[['survived']], test_size=0.2, random_state=41)
  model = LogisticRegression()
  model.fit(x_train, y_train)
  result = model.score(x_test, y_test)
  print("Random Seed:",random_seed,"Accuracy: %.3f%% (%.3f%%)" % (result.mean()*100.0, result.std()*100.0))

('Random Seed:', 7, 'Accuracy: 79.641% (0.000%)')
('Random Seed:', 42, 'Accuracy: 79.641% (0.000%)')
('Random Seed:', 43, 'Accuracy: 79.641% (0.000%)')
('Random Seed:', 91, 'Accuracy: 79.641% (0.000%)')
('Random Seed:', 52, 'Accuracy: 79.641% (0.000%)')


In [73]:
kfold = KFold(n_splits=5, random_state=42)
model = LogisticRegression()
results = model_selection.cross_val_score(model, x_train, y_train, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Accuracy: 78.527% (3.188%)


In [74]:
from sklearn.model_selection import KFold # import KFold
X = np.array(data_encoded.drop(['survived', 'Southampton'], axis=1)) # create an array
y = np.array(data_encoded['survived']) # Create another array

splits = [5,3,10]
for split in splits:
    kf = KFold(n_splits=split) # Define the split - into folds specified 
    kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
    print(kf) 
    KFold(n_splits=split, random_state=42, shuffle=False)
    train_index = []
    test_index = []
    sum = 0 
    print ("Number of Splits:",split)
    for train_index, test_index in kf.split(X):
      #print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X[train_index], X[test_index]
      Y_train, Y_test = y[train_index], y[test_index]
      model = LogisticRegression()
      model.fit(X_train, Y_train)
      result = model.score(X_test, Y_test)
      
      print("Accuracy: %.3f%%" % (result*100.0))
      sum = sum +(result)
    print ("Average accuracy:",(sum/split)*100)  

KFold(n_splits=5, random_state=None, shuffle=False)
('Number of Splits:', 5)
Accuracy: 78.443%
Accuracy: 78.443%
Accuracy: 76.647%
Accuracy: 75.301%
Accuracy: 81.325%
('Average accuracy:', 78.03188803116659)
KFold(n_splits=3, random_state=None, shuffle=False)
('Number of Splits:', 3)
Accuracy: 77.698%
Accuracy: 76.619%
Accuracy: 77.978%
('Average accuracy:', 77.43162870425681)
KFold(n_splits=10, random_state=None, shuffle=False)
('Number of Splits:', 10)
Accuracy: 76.190%
Accuracy: 80.952%
Accuracy: 76.190%
Accuracy: 79.518%
Accuracy: 75.904%
Accuracy: 78.313%
Accuracy: 77.108%
Accuracy: 74.699%
Accuracy: 80.723%
Accuracy: 79.518%
('Average accuracy:', 77.91164658634537)



There is a noticeable increase in validation accuracy when we are increasing the number of  folds from 3 to 5. As the number of folds is increased, the model is trained on a larger proportion of the data, and so should benefit from this up to a certain point.

The other trend is that as the folds increase, the proportion of the data used for validation decreases, and thus becomes less accurate as a statistical estimate of the model's ability to generalize to unseen data.

# Part Two: Hyperparameter Tuning

An important technique for improving the accuracy of a machine learning model is to undertake a process known as Hyperparameter Tuning or Hyperparameter Optimization. In order to understand this process, we first need to understand the difference between a model parameter and a model hyperparameter. 

### What is a model parameter?

A model parameter is a value that is generated by fitting our model to training data and is key to generating predictions with that model. They are **internal** to our model and we often are trying to estimate them as best as possible when we train the algorithm. 

For example, the parameters of a linear regression model would be its intercept value as well as the coefficient values on each of the X variables. Estimates of these crucial values (parameters) are obtained by fitting to the training data, perfectly define the model, are internal to the model, and are key to generating predictions. They are model parameters in every sense. 

### What is a model hyperparameter?

Hyperparameters are values that are key to how well our algorithm runs, yet are **external** to our model and cannot be estimated from the training process. They are more like settings for our algorithm which must be designated before it is run and impact its performance. Here is some further reading:

[Hyperparamters explanation on Quora](https://www.quora.com/What-are-hyperparameters-in-machine-learning)

[Jason Brownlee Article on the difference between Parameters and Hyperparameters](https://machinelearningmastery.com/difference-between-a-parameter-and-a-hyperparameter/)

### How do we find the best hyperparameters?

Since we can't learn the best hyperparameters for our model from the data, we essentially just pick values and see which ones lead to the highest accuracy. This can be a tedious and complex process especially for certain models like neural networks which can have dozens of hyperparameters. We will get you familiar with the process using a more simple logistic regression model. 

### How do you know what hyperparameters exist for your particular model? 

Most models/libraries have default hyperparameters that will be used if we don't specify them. In the model selection process you might try out multiple models on a dataset and see which one gets you the highest out-of-the-box performance, (using the default hyperparameters) and then pick a couple of the highest performing algorithms and attempt hyperparameter tuning on them to compare how different models benefit from this process. Once you have narrowed down the models that you would like to tune, a quick google search can tell you what hyperparameters exist for that algorithm. 

Often you can learn about potential hyperparameters by looking at the documentation for a given algorithm in a library, here's the documentation for sklearn's logistic regression, see if you can spot the hyperparameters:

[scikit-learn logistic regression docs](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)



## DO THIS: 

Lets hyperparameter tune our **titanic** predictions using 5-fold cross validation to compare the accuracy of our tuned models. 

### Manual Hyperparameter Tuning:

For our assignment today we are going to tune the 'C value' also known as the 'regularization strength' of our logistic regression as well as 'penalty' of our logistic regression algorithm.

Read up on the regularlization strength and penalty of a logistic regression function. What might be some good values to test out? Hint: Look at the parameter definitions on the sci-kit learn logistic regression documentation. 

[scikit-learn logistic regression docs](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)

[Regularization in Logistic Regression](https://www.kdnuggets.com/2016/06/regularization-logistic-regression.html)

Fit your model 5 different times using 5 different C values of your choosing. Which value gives the highest accuracy? 

There are only two penalty values that we can use. Evaluate the model two more times using each penalty once. Which penalty gives the highest accuracy?

In [76]:
# The sample code below uses the Pima Indans Diabetes Dataset. 
# Here we are setting the C value hyperparameter to 1 and the penalty hyperparameter to "l1". 
# You can designate your hyperparameters in a similar fashion.

import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

X = np.array(data_encoded.drop(['survived', 'Southampton'], axis=1)) # create an array
Y = np.array(data_encoded['survived'])
num_instances = len(X)
seed = 7
c_values = [1,3,5,7,100]
penality_values = ['l1','l2']
high = 0
pen = None
c_val = 0
for penality in penality_values:
  for c in c_values:
      print ("Penality = ",penality)
      print ("C ",c) 
      kfold = model_selection.KFold(n_splits=5, random_state=seed)
      model = LogisticRegression(C=c, penalty=penality) ##### This is the important line
      results = model_selection.cross_val_score(model, X, Y, cv=kfold)
      print(results)
      print ("Accuracy:",results.mean()*100)
      print ("\n")
      accu = results.mean()*100
      if (high < accu):
        high = accu
        pen = penality
        c_val = c
        
print ("penality with highest accuracy:",high)
print ("Best Penality:",pen)
print ("Best C val:",c_val)

('Penality = ', 'l1')
('C ', 1)
[0.77245509 0.77245509 0.76646707 0.75301205 0.80722892]
('Accuracy:', 77.43236418728807)


('Penality = ', 'l1')
('C ', 3)
[0.76646707 0.76646707 0.76646707 0.75903614 0.80722892]
('Accuracy:', 77.31332515691508)


('Penality = ', 'l1')
('C ', 5)
[0.77245509 0.76646707 0.75449102 0.77108434 0.81325301]
('Accuracy:', 77.55501046100571)


('Penality = ', 'l1')
('C ', 7)
[0.77245509 0.76646707 0.75449102 0.77108434 0.81325301]
('Accuracy:', 77.55501046100571)


('Penality = ', 'l1')
('C ', 100)
[0.77245509 0.76646707 0.75449102 0.77108434 0.80722892]
('Accuracy:', 77.43452853329487)


('Penality = ', 'l2')
('C ', 1)
[0.78443114 0.78443114 0.76646707 0.75301205 0.81325301]
('Accuracy:', 78.03188803116659)


('Penality = ', 'l2')
('C ', 3)
[0.77844311 0.77245509 0.76646707 0.76506024 0.79518072]
('Accuracy:', 77.55212466632999)


('Penality = ', 'l2')
('C ', 5)
[0.77844311 0.77245509 0.76646707 0.76506024 0.80120482]
('Accuracy:', 77.67260659404084)


('Pena

In [0]:
##### YOUR CODE HERE ##### - Feel free to add as many code cells as necessary.

### 1) Grid-Search Hyperparameter Tuning:

Imagine that your algorithm has 12 different potential hyperparameters and each them can take on 5 different values. Lets say that it takes your laptop 4 seconds to fit each fold of cross validation. For each 5-fold cross-validation it would then take 20 seconds to fit your model and get an accuracy reading reported back. Now imagine that you want to test every possible combination of hyperparameters on your algorithm to get the absolute highest accuracy. You can see how this might become exceedingly tedious and time-consuming to perform by hand. Some hyperparameters (like the C value) have much more than 5 potential values, making hyperparameter tuning a huge task. 

It is for this reason that more advanced optimization techniques exist, one of which we will be exploring today called GridSearch.

### What does GridSearch do?

GridSearch takes a dictionary of all of the different hyperparameters that you want to test, and then feeds all of the different combinations through the algorithm for you and then reports back to you which one had the highest accuracy. Pretty slick right? 

Here is some boilerplate code you can reference to create your implementations:

[Chris Albon Logistic Regression sklearn Hyperparameter Tuning with GridSearch](https://chrisalbon.com/machine_learning/model_selection/hyperparameter_tuning_using_grid_search/)

In [0]:
# Create logistic regression object

logistic = LogisticRegression()

In [0]:
# Create a list of all of the different penalty values that you want to test and save them to a variable called 'penalty'
penalty = ['l1', 'l2']

In [0]:
# Create a list of all of the different C values that you want to test and save them to a variable called 'C'
C = np.logspace(0,4,10)

In [50]:
# Now that you have two lists each holding the different values that you want test, use the dict() function to combine them into a dictionary. 
# Save your new dictionary to the variable 'hyperparameters'
# Print out the dictionary if you're curious as to what it euds up looking like.


hyperparameters = dict(C=C, penalty=penalty)

print (hyperparameters)


{'penalty': ['l1', 'l2'], 'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04])}


In [0]:
# Fit your model using gridsearch
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)
best_model = clf.fit(X, Y)

In [52]:
# Print the best penalty and C value from best_model.best_estimator_.get_params()
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

('Best Penalty:', 'l2')
('Best C:', 1.0)


In [53]:
# Print out all of the different combinations of your grid search values and their corresponding accuracies.
# https://stackoverflow.com/questions/22155953/how-to-print-out-an-accuracy-score-for-each-combination-within-gridsearch

from pprint import pprint
#pprint(clf.grid_scores_)

params, mean, std = [clf.cv_results_[key] for key in ['params', 'mean_test_score', 'std_test_score']]
pty = [p['penalty'] for p in params]
c = [p['C'] for p in params]

gridsearch = pd.DataFrame([pd.Series(x) for x in [pty, c, mean, std]]).T
gridsearch.columns = ['Penalty', 'C', 'Accuracy: Mean', 'Accuracy: Standard Deviation']

gridsearch

,Penalty,C,Accuracy: Mean,Accuracy: Standard Deviation
0,l1,1,0.77431,0.0243739
1,l2,1,0.779112,0.0271565
2,l1,2.78256,0.773109,0.024509
3,l2,2.78256,0.77431,0.0182685
4,l1,7.74264,0.77551,0.0229641
5,l2,7.74264,0.777911,0.0240033
6,l1,21.5443,0.77431,0.024157
7,l2,21.5443,0.77551,0.0229641
8,l1,59.9484,0.77431,0.024157
9,l2,59.9484,0.77431,0.024157


In [54]:
best_params = best_model.best_estimator_.get_params()
print('Best penalty: {}, Best C: {}'.format(best_params['penalty'], best_params['C']))

Best penalty: l2, Best C: 1.0


What hyperparameters give you the highest accuracy? Keep on testing diferent values and report the hyperparameters that give you the highest accuracy.

The best C is 1.0 and best penality is 'l2'
Let's add few more values to C and test this again inorder to find best C value.

In [0]:
C = np.linspace(1, 150)
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)
best_model = clf.fit(X, Y)

In [56]:
best_params = best_model.best_estimator_.get_params()
print('Best penalty: {}, Best C: {}'.format(best_params['penalty'], best_params['C']))

Best penalty: l2, Best C: 1.0


# Stretch Goals:

Explore more advanced automated approaches to hyperparameter tuning. Try and implemenet a random search approach: 

[Random Search Hyperparameter Tuning](https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/)

Then try a Bayesian Optimization Approach:

[Bayesian Optimization](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/)

[scikit-optimize](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

[optunity](http://optunity.readthedocs.io/en/latest/notebooks/notebooks/sklearn-automated-classification.html)

You could also try writing a blog post to show how well you understand Cross Validation or Hyperparameter Tuning, both are key concepts to practicing machine learning and would be valuable to demonstrate proficency in.


In [59]:
#Random Search Hyperparameter Tuning
# prepare a uniform distribution to sample for the alpha parameter

from sklearn.model_selection import RandomizedSearchCV
C = np.logspace(-1, 3, 1000)
penalty = ['l1', 'l2']
hyperparameters = dict(C=C, penalty=penalty)
# create and fit a ridge regression model, testing random alpha values
model = LogisticRegression()

rsearch =  RandomizedSearchCV(model, hyperparameters, n_iter=100, random_state=41)
rsearch.fit(X, Y)
print(rsearch.best_params_)


{'penalty': 'l1', 'C': 0.1259215613694151}


Random search finds even lower value of C. Let's try giving more values to C and test.

In [61]:
C = np.linspace(1, 200)
penalty = ['l1', 'l2']
hyperparameters = dict(C=C, penalty=penalty)

rsearch = RandomizedSearchCV(model, hyperparameters, n_iter=100, random_state=41)
rsearch.fit(X, Y)
print(rsearch.best_params_)

{'penalty': 'l2', 'C': 1.0}


The C value obtained is close to the C value generated in grid search 

**Bayesian Optimization**: This code is referred from https://github.com/thuijskens/bayesian-optimization/blob/master/python/gp.py 

In [0]:
#Bayesian Optimization
import sklearn.gaussian_process as gp

from sklearn.model_selection import cross_val_score
from scipy.stats import norm
from scipy.optimize import minimize

def expected_improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):
    """ expected_improvement
    Expected improvement acquisition function.
    Arguments:
    ----------
        x: array-like, shape = [n_samples, n_hyperparams]
            The point for which the expected improvement needs to be computed.
        gaussian_process: GaussianProcessRegressor object.
            Gaussian process trained on previously evaluated hyperparameters.
        evaluated_loss: Numpy array.
            Numpy array that contains the values off the loss function for the previously
            evaluated hyperparameters.
        greater_is_better: Boolean.
            Boolean flag that indicates whether the loss function is to be maximised or minimised.
        n_params: int.
            Dimension of the hyperparameter space.
    """

    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0

    return -1 * expected_improvement


def sample_next_hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):
    """ sample_next_hyperparameter
    Proposes the next hyperparameter to sample the loss function for.
    Arguments:
    ----------
        acquisition_func: function.
            Acquisition function to optimise.
        gaussian_process: GaussianProcessRegressor object.
            Gaussian process trained on previously evaluated hyperparameters.
        evaluated_loss: array-like, shape = [n_obs,]
            Numpy array that contains the values off the loss function for the previously
            evaluated hyperparameters.
        greater_is_better: Boolean.
            Boolean flag that indicates whether the loss function is to be maximised or minimised.
        bounds: Tuple.
            Bounds for the L-BFGS optimiser.
        n_restarts: integer.
            Number of times to run the minimiser with different starting points.
    """
    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))

        if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x


def bayesian_optimisation(n_iters, sample_loss, bounds, x0=None, n_pre_samples=10,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7):
    """ bayesian_optimisation
    Uses Gaussian Processes to optimise the loss function `sample_loss`.
    Arguments:
    ----------
        n_iters: integer.
            Number of iterations to run the search algorithm.
        sample_loss: function.
            Function to be optimised.
        bounds: array-like, shape = [n_params, 2].
            Lower and upper bounds on the parameters of the function `sample_loss`.
        x0: array-like, shape = [n_pre_samples, n_params].
            Array of initial points to sample the loss function for. If None, randomly
            samples from the loss function.
        n_pre_samples: integer.
            If x0 is None, samples `n_pre_samples` initial points from the loss function.
        gp_params: dictionary.
            Dictionary of parameters to pass on to the underlying Gaussian Process.
        random_search: integer.
            Flag that indicates whether to perform random search or L-BFGS-B optimisation
            over the acquisition function.
        alpha: double.
            Variance of the error term of the GP.
        epsilon: double.
            Precision tolerance for floats.
    """

    x_list = []
    y_list = []

    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(sample_loss(params))
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(sample_loss(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for n in range(n_iters):

        model.fit(xp, yp)

        # Sample next hyperparameter
        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * expected_improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = sample_next_hyperparameter(expected_improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        # Sample loss for new set of parameters
        cv_score = sample_loss(next_sample)

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

In [0]:
def sample_loss(params):
    C = params[0]
    penalty = 'l1' if params[1] < (bounds[1,1]//2) else 'l2'
    
    model = LogisticRegression(C=C, penalty=penalty)
    
    return cross_val_score(model,
                           X=X,
                           y=Y,
                           cv=5).mean()

In [0]:
n_iters = 10
bounds = np.array([[0, 10], [0, 1]])
hyperparameters, scores = bayesian_optimisation(n_iters, sample_loss, bounds)

In [70]:
best_ix = np.argmax(scores)
print(best_ix)
best_C = hyperparameters[best_ix,0]
best_penalty = 'l1' if hyperparameters[best_ix,1] < 0.5 else 'l2'
print('Best penalty: {}, Best C: {}'.format(best_penalty, best_C))

5
Best penalty: l2, Best C: 0.78963598403
